In [2]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from typing import List, Tuple, Dict, Union, Any, Optional
from tqdm import tqdm
import json
try:
    os.chdir(os.path.join(os.getcwd(), '../../Language-Technology-Project'))
    print(os.getcwd())
except:
    print("ALready in current dir")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/moassaf/Documents/Development/Organisations/RUG/Repos/Language-Technology-Project


### Load the data


In [3]:
path = "./datasets/threads.jsonl"

def read_data(path: str) -> dict:
    # parse the content of the json file
    text = open(path, 'r').read()
    data = json.loads(text)
    return data

data = read_data(path)